# BioExplorer - Astrocytes
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, Vector3

# url = 'r4i0n15:5000'
url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

In [ ]:
population_name = 'astrocytes'
vasculature_population_name = 'vasculature'

'''Defines a clipping cylinder with an origin, direction and radius'''
o = Vector3(334.274978637695, 0, 598.822242736816)
d = Vector3(0.0, 1.0, 0.0)
r = 100.0
filter = 'public.length(public.cross(array[%f-x,%f-y,%f-z], array[%f::double precision, %f, %f])) < %f' % (o.x, o.y, o.z, d.x, d.y, d.z, r)

print(filter)

### Load astrocytes

In [ ]:
assembly_name = 'Astrocytes'
be.remove_assembly(assembly_name)
vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_astrocytes(
    assembly_name=assembly_name,
    population_name='Astrocytes',
    vasculature_population_name=vasculature_population_name,
    # population_color_scheme=be.POPULATION_COLOR_SCHEME_ID,
    radius_multiplier=1.0,
    load_micro_domain=True,
    # load_dendrites=False,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    micro_domain_representation=be.MICRO_DOMAIN_REPRESENTATION_CONVEX_HULL,
    realism_level=be.MORPHOLOGY_REALISM_LEVEL_END_FOOT,
    # realism_level=be.MORPHOLOGY_REALISM_LEVEL_DENDRITE + be.MORPHOLOGY_REALISM_LEVEL_SOMA,
    sql_filter=filter
    # sql_filter='guid=7178'
)

In [ ]:
# assembly_name = 'Vasculature'
# be.remove_assembly(assembly_name)
# vasculature_assembly = be.add_assembly(assembly_name)
# vasculature_model = be.add_vasculature(
#     assembly_name=assembly_name,
#     population_name=vasculature_population_name,
#     representation=be.VASCULATURE_REPRESENTATION_OPTIMIZED_SEGMENT,
#     # color_scheme=be.VASCULATURE_COLOR_SCHEME_REGION,
#     # realism_level=be.VASCULATURE_REALISM_LEVEL_ALL,
#     sql_filter=filter
# )

In [ ]:
status = be.reset_camera()
# status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [ ]:
status = core.set_renderer(
    background_color=[0, 0, 0],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 100.0
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 40.0
# params.max_bounces = 10
params.max_bounces = 1
params.show_background = False
params.exposure = 1.0
status = core.set_renderer_params(params)
status = core.set_renderer()

In [ ]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    palette = sns.color_palette(palette_name, be.NB_MATERIALS_PER_MORPHOLOGY)
    shading_mode = be.SHADING_MODE_DIFFUSE

    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        color = palette[mid]
        if mid == be.ASTROCYTE_MATERIAL_MITOCHONDRION:
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.25)
        elif mid == be.ASTROCYTE_MATERIAL_NUCLEUS:
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.ASTROCYTE_MATERIAL_SOMA:
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.ASTROCYTE_MATERIAL_END_FOOT:
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        elif mid == be.ASTROCYTE_MATERIAL_MICRO_DOMAIN:
            # opacities.append(0.25)
            opacities.append(1.0)
            colors.append([0.5, 0.5, 0.5])
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            colors.append(color)
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            glossinesses.append(1.0)
            user_params.append(3.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

import seaborn as sns
model_ids = be.get_model_ids()['ids']
nb_models = len(model_ids)
for model_id in model_ids:
    set_morphology_materials(model_id, 'Set1')
status = core.set_renderer()